In [1]:
import os
# ALWAYS COMMIT CUDA DEVICES EMPTY
# Environment declarations for adding GPUs to process
# If devices string is changed, must restart kernal to use new configuration
os.environ["CUDA__DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import pandas as pd
import numpy as np

from datetime import datetime
from time import sleep
from math import ceil
from collections import Counter
from random import shuffle
import matplotlib.pyplot as plt
import cv2
import gc

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Lambda, \
                                Flatten, BatchNormalization, GlobalAveragePooling2D
from keras import optimizers

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.utils.vis_utils import plot_model

import tensorflow as tf
import random

import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
image_crops = 'images/cropped'

# Gets all full paths for cropped images in images folder and video frame folder
def join_image_paths(): # Try train/validate on smaller set (only image crops)
    # Look into cropping images and rescaling so faces are most of 224x244
    paths = [image_crops]
    allPaths = []
    
    # Iter through roots, iter through files in a root then append paths list with full path
    for onePath in paths:
        p = os.listdir(onePath)
        shuffle(p)
        for oneFile in p:
            allPaths.append(os.path.join(onePath, oneFile))
            
    # Return list of all full paths
    return allPaths

# Function to format all filenames in 'image' column to only have subject number
# Useful when replacing image names and correlating image with full path
def format_subs(DF):
    DF['image'] = DF['image'].apply(lambda c: c.split('_')[0])
    return DF

# Function to replace image names in DF with full path to image
def replace_image_names(paths, DF):
    print('Replacing filenames with full paths')
    new_df = pd.DataFrame(columns=('path', 'ethnicity'))
    df = format_subs(DF)
    
    lastTime = datetime.now()
    for i, onePath in enumerate(paths):
        if not yuanjie_crops in onePath:
            fileSub = '-'.join(onePath.split('/')[-1].split('-')[2:]).split('_')[0]
        else:
            fileSub = '.'.join(onePath.split('/')[-1].split('_')[:2])
            
        fileEth = df[df['image']==fileSub]['ethnicity'].iloc[0]
        
        new_df.loc[i] = [onePath, fileEth]
        
        if i%10000==0:
            if i == 0:
                step = 1
            else:
                step = 10000
            print(i, 'of', len(paths))
            print('Estimated time remaining:', (datetime.now()-lastTime)*((len(paths)-(i+1))/(step)))
            lastTime = datetime.now()
    
    print('\nEthnicity Counts...')
    for oneLabel in poss_labels:
        print('%s: %s' % (oneLabel, new_df[new_df['ethnicity'] == oneLabel].shape[0]))
    print('\n')
        
    return new_df

# Arr is list of labels
# Key is sorted de-duplicated list of labels
def getOneHot(arr, key):
    print('Processing one-hot encoding...')
    output = np.zeros((arr.shape[0], len(key)))
    for i, elem in enumerate(arr):
        output[i][key.index(elem)] = 1
    return output

def split_sets(DF):
    train, validate, test = np.split(DF.sample(frac=1).reset_index(drop=True), [int(.6*len(DF)), int(.8*len(DF))])
    train = train.reset_index(drop=True)
    validate = validate.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    train.to_csv(training_path, index=False)
    validate.to_csv(validation_path, index=False)
    test.to_csv(testing_path, index=False)
    
    DF.to_csv(labels_path, index=False)
    
    return train, validate, test

def conv_oneHot(DF):
    DF['ethnicity'] = DF['ethnicity'].apply(lambda c: np.fromstring(c.replace('[', '')
                                                        .replace(']', '').replace(',', ''), sep=' '))
    
    return DF

def str_to_oneHot(string):
    string = string.replace('[', '').replace(']', '').replace(',', '')
    out_list = np.fromstring(string, dtype=np.float, sep=',')
    
    return out_list

def remove_some_eth(df, label, reduce):
    df['ethnicity'] = df['ethnicity']
    base = df[df['ethnicity'] != label].reset_index(drop=True)
    eth_to_drop = df[df['ethnicity'] == label].reset_index(drop=True)
    eth = eth_to_drop.iloc[:int(eth_to_drop.shape[0]*reduce)]
    
    new_df = pd.concat([base, eth]).sample(frac=1).reset_index(drop=True)
    print('Ethnicity %s reduced... new counts...' % label)
    for l in sorted(set(new_df['ethnicity'].tolist())):
        print('%s: %s' % (l, new_df[new_df['ethnicity']==l].shape[0]))
    return new_df
    

In [ ]:
# Gets ethnicity data for cropped images and inits CSV output paths
ethPath = 'subData/ethLabels.csv'
ethData = pd.read_csv(ethPath, index_col=False)
training_path = 'subData/train.csv'
validation_path = 'subData/validation.csv'
testing_path = 'subData/testing.csv'
labels_path = 'subData/labels.csv'
    
poss_labels = sorted(set(ethData['ethnicity'].tolist()))
cropPaths = join_image_paths()

n = datetime.now()
choice = 1
if choice == 1:
    print('Processing labels...')
    # Resets data in output files by re-processing all data... Takes longer than choice 2
    ethData = remove_some_eth(replace_image_names(cropPaths, ethData), label='Caucasian', reduce=0.6)
    ethData['ethnicity'] = list(getOneHot(np.array(ethData['ethnicity'].tolist()), poss_labels))
    training_set, validation_set, testing_set = split_sets(ethData)
    print('Total time to complete:', datetime.now()-n)
elif choice == 2:
    print('Loading data from CSVs...')
    training_set = conv_oneHot(pd.read_csv(training_path))
    validation_set = conv_oneHot(pd.read_csv(validation_path))
    testing_set = conv_oneHot(pd.read_csv(testing_path))
    ethData = pd.read_csv(labels_path)
    print('Total time to complete:', datetime.now()-n)
else:
    print('Invalid choice...')
    
print('\nTraining, validation and testing sets loaded...')
print('Sizes:\nTraining: %s\nValidation: %s\nTesting: %s' % (training_set.shape[0], validation_set.shape[0],
                                                            testing_set.shape[0]))

In [ ]:
ethData['ethnicity'] = ethData['ethnicity'].astype(str)
for oneLabel in sorted(set(ethData['ethnicity'].values.tolist())):
    l = []
    for char in oneLabel:
        try:
            l.append(float(char))
        except:
            pass
        
    print('\n', poss_labels[np.argmax(l)], 'label:', oneLabel)
    print('Train set instances: ', training_set[training_set['ethnicity'].apply(lambda c: np.argmax(c))==np.argmax(l)].shape[0])
    print('Validation set instances: ', validation_set[validation_set['ethnicity'].apply(lambda c: np.argmax(c))==np.argmax(l)].shape[0])
    print('Test set instances: ', testing_set[testing_set['ethnicity'].apply(lambda c: np.argmax(c))==np.argmax(l)].shape[0])

In [ ]:
def getAcc(model, X_set, Y_set, incorrect_dict):
    preds = model.predict(X_set)
    if not incorrect_dict:
        incorrect_dict = {key: [] for key in poss_labels}
        
    correct = 0
    for i, prediction in enumerate(preds):
        if np.argmax(prediction) == np.argmax(Y_set[i]):
            correct += 1
        else:
            actu_eth = poss_labels[np.argmax(Y_set[i])]
            pred_eth = poss_labels[np.argmax(prediction)]
            incorrect_dict[actu_eth].append(pred_eth)
            
    return correct/len(preds), incorrect_dict

def mostCommonIncorrectPred(incorrect_dict):
    outStr = '\n\nActual Ethnicity: Most Common Misprediction\n'
    outCSV = 'ACT_LABEL,' + ','.join(poss_labels) + '\n'
    for key in poss_labels:
        amount = []
        outCSV += key + ','
        for lookFor in poss_labels:
            amount.append(incorrect_dict[key].count(lookFor))
            outCSV += str(amount[-1]) + ','
            
        outCSV += '\n'
        outStr += '%s: %s\n' % (key, poss_labels[np.argmax(amount)])
        
    return outStr, outCSV
            

def getMaxBatch(DF, max_set):
    current_batch = pd.DataFrame(columns=(list(DF)))
    DF = DF.reset_index(drop=True)
    
    for i, row in DF.iterrows():
            
        current_batch.loc[i] = row
        
        if int(i) == max_set-1:
            break
        
    DF = DF.drop(DF.index[:i+1])
    
    return DF.reset_index(drop=True), current_batch.reset_index(drop=True)

def load_images(image_paths):
    size = 224
    all_images = np.zeros((len(image_paths), size, size, 3))
    for i, _path in enumerate(image_paths):
        # Read the image
        img = plt.imread(_path)
        # Resize the image
        img = cv2.resize(img,(size,size))
        # Normalize the image
        img = np.divide(img,[255.,255., 255.])
        # Set into X_tr
        all_images[i] = img
        
    return all_images

file = 'logs/printout.txt'
if not os.path.exists('/'.join(file.split('/')[:-1])):
    os.mkdir(file)
def printO(string, filename=file, header=True, custom_header=False, new_line=True):
    if string=='CLEARFILE':
        f = open(filename, 'w')
        if custom_header:
            fill = custom_header
        elif header:
            fill = 'Training began at: %s\n\n' % datetime.now()
        else:
            fill = ''
        f.write(fill)
        f.close()
        print('User Output File %s Cleared...' % filename.split('/')[-1])
    else:
        print(string)
        f = open(filename, 'a')
        if new_line:
            string += '\n'
        f.write(string)
        f.close()


In [ ]:
imageGen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

In [2]:
# Create model off of VGG16 skipping top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add average pulling layer (test results and also allows for CAM output)
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layer after average pulling
x = Dense(1024, activation='relu')(x)

# Add softmax for predictions
#predictions = Dense(len(poss_labels), activation='softmax')(x)
predictions = Dense(4, activation='softmax')(x)
# Set inputs (default) and outputs (new softmax layer)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile with SGD Momentum, low learning rate and categorical... consider lowering momentum
# Use model.evaluate (look up docs) for training loss, also think about increasing learning rate
model.compile(optimizer=optimizers.SGD(lr=0.0005, momentum=0.9), loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])
print('Base model ready for training...')
model.summary()
plot_model(model, to_file='VGG16_model_plot.png', show_shapes=True, show_layer_names=True)

Base model ready for training...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56,

In [ ]:
epochs = 50
image_max = 3200
batch_size = 32 # 3200 / 32 = 100
allAcc = []

log_base = 'logs'
if not os.path.exists(log_base):
    os.mkdir(log_base)
if not os.path.exists('models'):
    os.mkdir('models')
# Set base path names and clear files of contents
base = 'models/ethnicity/'
if not os.path.exists(base):
    os.mkdir(base)
    
metrics_path = os.path.join(log_base, 'ethModelMetrics.csv')
incorrect_preds_path = os.path.join(log_base, 'incorrectPredictionMetric.csv')

printO('CLEARFILE')
printO('CLEARFILE', filename=metrics_path, custom_header='EPOCH,TR_ACC,TR_LOSS,VAL_ACC,VAL_LOSS\n')
printO('CLEARFILE', filename=incorrect_preds_path, header=False)

# Create tickers for time estimates
ticker = datetime.now()
epoch_tick = datetime.now()
epoch_metrics = {'train': [], 'validation': []} # Metrics list for loss and accuracy to be stored in

# Try-except loop for logging of any errors
try:
    for i in range(epochs):
        # Print user output to file
        printO('\n' + '='*30)
        printO('Epoch %s of %s' %(i+1, epochs))
        printO('='*30 + '\n')

        # Reset temp training set which is continuously made smaller by getMaxBatch func
        print('Training...')
        temp_train = training_set

        # Init counter and metric lists
        c = 0
        acc = []
        loss = []
        incorrect_dict = False
        while temp_train.shape[0] > 0: # Init while loop to check if temp train set is greater than 0
            # Take current batch out of temp train batch of amount image_max then return resulting batches
            temp_train, current_batch = getMaxBatch(temp_train, image_max)

            # User output, loads X batch images and gets Y preds from current batch
            print('Loading current batch of %s images into memory...' % current_batch.shape[0])
            X_train = load_images(current_batch['path'].tolist())
            Y_train = current_batch['ethnicity'].tolist()

            # Inits batch time estimator and batch counter
            batchTime = datetime.now()
            batches = 0

            # Cycles through training X and Y sets in sizes of batch_size
            for X_batch, Y_batch in imageGen.flow(X_train, Y_train, batch_size=batch_size):
                # Fits current batch to model, increments model
                model.fit(X_batch, Y_batch, verbose=0)
                batches += 1
                if batches >= len(X_train) / batch_size:
                    # Break the generator before it breaks us
                    # AKA, generator will do infinite loop if we don't break
                    break

                # Updates user and metric information... gives time estimate on train batch completion
                elif batches % 10 == 0:
                    a, incorrect_dict = getAcc(model, X_batch, Y_batch, incorrect_dict)
                    acc.append(a)
                    loss.append(model.evaluate(X_batch, Y_batch, verbose=0))
                    printO("Training - Batch : %s" % batches),
                    printO("Training - of batch %s" % (len(X_train) // batch_size))
                    printO("Estimated time for training batch completion: %s" % (((datetime.now()-batchTime)/10)*
                                                                        ((len(X_train)//batch_size)-batches)))
                    batchTime = datetime.now()

            # Increments counter, more user output
            c += 1
            printO('Training - At iteration %s of %s for batches' % (c, ceil(training_set.shape[0]/image_max)))
            printO('Estimated time for this epoch\'s training completion: %s\n' %((datetime.now()-ticker)*ceil(
                                                                                temp_train.shape[0]/image_max)))
            ticker = datetime.now()

        # Updates epoch metrics and prints them to file
        epoch_metrics['train'].append([np.mean(acc)*100, np.mean(loss)])
        met = '%s,%s,%s,' % (i+1, np.mean(acc)*100, np.mean(loss))
        printO(met, filename=metrics_path, new_line=False)
        printO('Training mean of accuracy and loss for this epoch: %s' % epoch_metrics['train'][-1])

        # Resets variables for validation stage
        incorrect_dict = False
        temp_validation = validation_set
        ticker = datetime.now()
        c = 0
        acc = []
        loss = []
        print('Validating...')

        while temp_validation.shape[0] > 0: # Init while loop to check if temp validation set is greater than 0
            # Take current batch out of temp validatin batch of amount image_max then return resulting batches
            temp_validation, current_batch = getMaxBatch(temp_validation, image_max)

            # User output, loads X batch images and gets Y preds from current batch
            X_validation = load_images(current_batch['path'].tolist())
            Y_validation = current_batch['ethnicity'].tolist()
            batchTime = datetime.now()
            batches=0

            # Cycles through training X and Y sets in sizes of batch_size
            for X_batch, Y_batch in imageGen.flow(X_validation, Y_validation, batch_size=batch_size):
                # Gets accuracy of validation set
                a, incorrect_dict = getAcc(model, X_batch, Y_batch, incorrect_dict)
                acc.append(a)
                batches += 1

                if batches >= len(X_validation) / batch_size:
                    # Break the generator before it breaks us
                    # AKA, generator will do infinite loop if we don't break
                    break

                # Ticker/user output
                elif batches % 10 == 0:
                    loss.append(model.evaluate(X_batch, Y_batch, verbose=0))
                    printO("Validation - Batch : %s" % batches),
                    printO("Validation - of batch %s" % (len(X_validation) // batch_size))
                    printO("Estimated time for validation batch completion: %s" % (((datetime.now()-batchTime)/10)*
                                                                        ((len(X_validation)/batch_size)-batches)))
                    batchTime = datetime.now()

            c += 1
            printO('Validation - At iteration %s of %s for batches' % (c, ceil(validation_set.shape[0]/image_max)))
            printO('Estimated time for this epoch\'s validation completion: %s\n' %((datetime.now()-ticker)*ceil(
                                                                                temp_validation.shape[0]/image_max)))
            ticker = datetime.now()

        printO('\n')
        # Epoch metrics updated and output
        epoch_metrics['validation'].append([np.mean(acc)*100, np.mean(loss)])
        met = '%s,%s' % (np.mean(acc)*100, np.mean(loss))
        printO(met, filename=metrics_path)
        printO('Validation mean of accuracy and loss for this epoch: %s' % epoch_metrics['validation'][-1])

        # Epoch accuracy and metrics information output if current model is best model
        allAcc.append(np.mean(acc)*100)
        s, outCSV = mostCommonIncorrectPred(incorrect_dict)
        printO(outCSV, filename=incorrect_preds_path)
        if max(allAcc) == allAcc[-1]:
            model_path = os.path.join(base, 'ethnicity_acc_%.2f_.h5' % allAcc[-1])
            log_path = os.path.join(base, 'LOG-ethnicity_acc_%.2f.txt' % allAcc[-1])
            printO('CLEARFILE', filename=log_path)
            outStr = 'LOG INFO FOR ETHNICITY TRAINING BEST MODEL FOUND\n\nEPOCH: %s\nACCURACY: %s\nLOSS: %s' % (
                                i+1, allAcc[-1], np.mean(loss))
            outStr += s
            printO(outStr, filename=log_path)

            for _file in os.listdir('/'.join(model_path.split('/')[:-1])):
                if 'ethnicity_acc' in _file:
                    os.remove(os.path.join(base, _file))

            model.save(model_path)

        printO('Accuracy of epoch: %.2f' % (np.mean(acc)*100))
        printO('Time for epoch: %s' % (datetime.now()-epoch_tick))
        printO('As of %s; estimated time remaining for training/validation: %s' % (datetime.now(), 
                                            (datetime.now()-epoch_tick)*(epochs-(i+1))))
        epoch_tick = datetime.now()
        
except Exception as e:
    printO('MODEL TRAINING FAILURE\n\nERROR:\n%s' % e)